In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.8.2'

In [3]:
import pickle

In [4]:
file = open("/content/drive/MyDrive/Software Project/X_normalized.pickle", "rb")
X = pickle.load(file)
file = open("/content/drive/MyDrive/Software Project/Y.pickle", "rb")
Y = pickle.load(file)

In [14]:
X.shape

(1009, 179, 60, 60, 1)

In [6]:
X = X.reshape(1009, 179, 60, 60, 1)
X.shape

(1009, 179, 60, 60, 1)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 5)

In [9]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((706, 179, 60, 60, 1), (706,), (303, 179, 60, 60, 1), (303,))

In [10]:
from tensorflow.keras import Input
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Dense, Flatten, LSTM, TimeDistributed, Reshape
from tensorflow.keras import Model
from tensorflow.keras import callbacks

In [11]:
import numpy as np
np.unique(Y)

array([0, 1, 2, 3, 4])

In [12]:
EPOCHS = 50
BATCH_SIZE = 16
callbacks_params = [
             callbacks.EarlyStopping(monitor='loss', patience=10),
             callbacks.ReduceLROnPlateau(verbose=1),
             callbacks.ModelCheckpoint('/content/drive/MyDrive/Software Project/weights.{epoch:02d}-{val_loss:.2f}.hdf5', verbose=1),
]

### Model Architecture

In [13]:
cnn_input= Input((179, 60, 60, 1))   #Frames,height,width
conv1 = TimeDistributed(Conv2D(32, kernel_size=(5,5), activation='relu'))(cnn_input)
first_BatchNormalization = BatchNormalization()(conv1)

conv2 = TimeDistributed(Conv2D(32, kernel_size=(5,5), activation='relu'))(first_BatchNormalization)
second_BatchNormalization = BatchNormalization()(conv2)

pool1=TimeDistributed(MaxPooling2D(pool_size=(4,4)))(second_BatchNormalization)
flat=TimeDistributed(Flatten())(pool1)
cnn_op= TimeDistributed(Dense(100))(flat)

lstm = LSTM(128, return_sequences=True, activation='tanh')(cnn_op)
dense_layer =TimeDistributed(Dense(100))(lstm)
output_layer = Dense(5, activation = "softmax")(dense_layer)
model = Model(inputs=[cnn_input], outputs=output_layer)

In [15]:
model.compile(optimizer = tf.keras.optimizers.Adam(), loss = tf.keras.losses.CategoricalCrossentropy(), metrics = [tf.keras.metrics.Accuracy()])

In [ ]:
model.fit(X_train, Y_train, batch_size = BATCH_SIZE, callbacks = callbacks_params, epochs = EPOCHS, verbose = 1)